In [11]:


from sklearn.cross_validation import train_test_split
import pandas as pd
import xgboost as xgb

random_seed = 1225

#set data path
train_x_csv = "C:/Users/tianjiayang/loan_predicting/data/train_x.csv"
train_y_csv = "C:/Users/tianjiayang/loan_predicting/data/train_y.csv"
test_x_csv = "C:/Users/tianjiayang/loan_predicting/data/test_x.csv"
features_type_csv = "C:/Users/tianjiayang/loan_predicting/data/features_type.csv"

#load data
train_x = pd.read_csv(train_x_csv)
train_y = pd.read_csv(train_y_csv)
train_xy = pd.merge(train_x,train_y,on='uid')

test = pd.read_csv(test_x_csv)
test_uid = test.uid
test_x = test.drop(['uid'],axis=1)

#dictionary {feature:type}
features_type = pd.read_csv(features_type_csv)
features_type.index = features_type.feature
features_type = features_type.drop('feature',axis=1)
features_type = features_type.to_dict()['type']


feature_info = {}
features = list(train_x.columns)
features.remove('uid')

for feature in features:
    max_ = train_x[feature].max()
    min_ = train_x[feature].min()
    n_null = len(train_x[train_x[feature]<0])  #number of null
    n_gt1w = len(train_x[train_x[feature]>10000])  #greater than 10000
    feature_info[feature] = [min_,max_,n_null,n_gt1w]

#see how many neg/pos sample
print ("neg:{0},pos:{1}".format(len(train_xy[train_xy.y==0]),len(train_xy[train_xy.y==1])))


#split train set,generate train,val,test set
train_xy = train_xy.drop(['uid'],axis=1)
# 20% for validation
train,val = train_test_split(train_xy, test_size = 0.2,random_state=1)#random_state is of big influence for val-auc
y = train.y
X = train.drop(['y'],axis=1)
val_y = val.y
val_X = val.drop(['y'],axis=1)



#xgboost start here
dtest = xgb.DMatrix(test_x)
dval = xgb.DMatrix(val_X,label=val_y)
dtrain = xgb.DMatrix(X, label=y)
params={
    #gbtree,gblinear,dart
	'booster':'gbtree',
    #
	'objective': 'binary:logistic',
    #100轮后指标不上升提前终止
    
	'early_stopping_rounds':100,
    #Control the balance of positive and negative weights, useful for unbalanced classes. 
	'scale_pos_weight': 1500.0/13458.0,
    #
    'eval_metric': 'auc',
    #Minimum loss reduction required to make a further partition on a leaf node of the tree. 
    #The larger gamma is, the more conservative the algorithm will be.
	'gamma':0.1,#0.2 is ok
    #Maximum depth of a tree.
	'max_depth':8,
    #L2 reg
	'lambda':550,
    #Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees.
    'subsample':0.7,
    #Subsample ratio of columns when constructing each tree.
    'colsample_bytree':0.3,
    #Min sum of instance weight needed in a child. If the tree partition step results in a leaf node with the sum of instance weig
    'min_child_weight':2.5, 
    #learning rate
    'eta': 0.007,
	'seed':random_seed,
	'nthread':7
    }

watchlist  = [(dtrain,'train'),(dval,'val')]#The early stopping is based on last set in the evallist
model = xgb.train(params,dtrain,num_boost_round=15000,evals=watchlist)
#Save the model to a file.
model.save_model('./model/xgb.model')
print ("best best_ntree_limit{}".format(model.best_ntree_limit))  

#predict test set (from the best iteration)
test_y = model.predict(dtest,ntree_limit=model.best_ntree_limit)
test_result = pd.DataFrame(columns=["uid","score"])
test_result.uid = test_uid
test_result.score = test_y
test_result.to_csv("xgb.csv",index=None,encoding='utf-8')  #remember to edit xgb.csv , add ""


#save feature score and feature information:  feature,score,min,max,n_null,n_gt1w
feature_score = model.get_fscore()
for key in feature_score:
    feature_score[key] = [feature_score[key]]+feature_info[key]+[features_type[key]]

feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1},{2},{3},{4},{5},{6}\n".format(key,value[0],value[1],value[2],value[3],value[4],value[5]))

with open('feature_score.csv','w') as f:
    f.writelines("feature,score,min,max,n_null,n_gt1w\n")
    f.writelines(fs)





neg:1542,pos:13458
[23:15:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 16 pruned nodes, max_depth=8
[0]	train-auc:0.64721	val-auc:0.603096
[23:15:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 18 pruned nodes, max_depth=8
[1]	train-auc:0.668453	val-auc:0.607658
[23:15:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 36 pruned nodes, max_depth=8
[2]	train-auc:0.683845	val-auc:0.629086
[23:15:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 26 pruned nodes, max_depth=8
[3]	train-auc:0.685286	val-auc:0.628117
[23:15:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 24 pruned nodes, max_depth=8
[4]	train-auc:0.691175	val-auc:0.629923
[23:15:19] C:\Users\Ad

[23:15:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 16 pruned nodes, max_depth=8
[43]	train-auc:0.708502	val-auc:0.642311
[23:15:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 26 pruned nodes, max_depth=8
[44]	train-auc:0.708576	val-auc:0.641972
[23:15:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 40 pruned nodes, max_depth=8
[45]	train-auc:0.708676	val-auc:0.641922
[23:15:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 20 pruned nodes, max_depth=8
[46]	train-auc:0.708549	val-auc:0.641764
[23:15:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 26 pruned nodes, max_depth=8
[47]	train-auc:0.708653	val-auc:0.642136
[23:15:29] C:\Users\Administrator\D

[23:15:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 30 pruned nodes, max_depth=8
[86]	train-auc:0.710176	val-auc:0.643659
[23:15:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 20 pruned nodes, max_depth=8
[87]	train-auc:0.710146	val-auc:0.643811
[23:15:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 22 pruned nodes, max_depth=8
[88]	train-auc:0.710126	val-auc:0.644053
[23:15:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 22 pruned nodes, max_depth=8
[89]	train-auc:0.710119	val-auc:0.644113
[23:15:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 22 pruned nodes, max_depth=8
[90]	train-auc:0.710038	val-auc:0.644089
[23:15:36] C:\Users\Administrator\D

[23:15:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 32 pruned nodes, max_depth=8
[129]	train-auc:0.712331	val-auc:0.644866
[23:15:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 30 pruned nodes, max_depth=8
[130]	train-auc:0.712362	val-auc:0.645187
[23:15:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 28 pruned nodes, max_depth=8
[131]	train-auc:0.712492	val-auc:0.645561
[23:15:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 34 pruned nodes, max_depth=8
[132]	train-auc:0.712451	val-auc:0.645592
[23:15:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 30 pruned nodes, max_depth=8
[133]	train-auc:0.712413	val-auc:0.645837
[23:15:44] C:\Users\Administra

[23:15:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 26 pruned nodes, max_depth=8
[172]	train-auc:0.713707	val-auc:0.649303
[23:15:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 26 pruned nodes, max_depth=8
[173]	train-auc:0.713765	val-auc:0.649483
[23:15:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 28 pruned nodes, max_depth=8
[174]	train-auc:0.713692	val-auc:0.649569
[23:15:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 36 pruned nodes, max_depth=8
[175]	train-auc:0.713733	val-auc:0.649553
[23:15:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 28 pruned nodes, max_depth=8
[176]	train-auc:0.713936	val-auc:0.650076
[23:15:52] C:\Users\Administra

...

[00:04:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 56 pruned nodes, max_depth=8
[14795]	train-auc:0.974662	val-auc:0.718416
[00:04:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 48 pruned nodes, max_depth=8
[14796]	train-auc:0.974667	val-auc:0.718418
[00:04:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 48 pruned nodes, max_depth=8
[14797]	train-auc:0.974671	val-auc:0.718426
[00:04:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 52 pruned nodes, max_depth=8
[14798]	train-auc:0.974671	val-auc:0.718447
[00:04:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 34 pruned nodes, max_depth=8
[14799]	train-auc:0.974677	val-auc:0.718439
[00:04:10] C:\Users\

[00:04:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 52 pruned nodes, max_depth=8
[14837]	train-auc:0.974822	val-auc:0.718429
[00:04:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 44 pruned nodes, max_depth=8
[14838]	train-auc:0.974827	val-auc:0.718434
[00:04:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 52 pruned nodes, max_depth=8
[14839]	train-auc:0.974828	val-auc:0.718423
[00:04:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 56 pruned nodes, max_depth=8
[14840]	train-auc:0.974832	val-auc:0.718442
[00:04:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 56 pruned nodes, max_depth=8
[14841]	train-auc:0.974834	val-auc:0.718449
[00:04:22] C:\Users\

[00:04:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 48 pruned nodes, max_depth=8
[14879]	train-auc:0.97497	val-auc:0.718464
[00:04:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 60 pruned nodes, max_depth=8
[14880]	train-auc:0.974976	val-auc:0.718464
[00:04:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 50 pruned nodes, max_depth=8
[14881]	train-auc:0.974979	val-auc:0.71846
[00:04:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 52 pruned nodes, max_depth=8
[14882]	train-auc:0.974985	val-auc:0.718462
[00:04:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 40 pruned nodes, max_depth=8
[14883]	train-auc:0.974985	val-auc:0.718451
[00:04:33] C:\Users\Ad

[00:04:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 44 pruned nodes, max_depth=8
[14921]	train-auc:0.975109	val-auc:0.718442
[00:04:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 54 pruned nodes, max_depth=8
[14922]	train-auc:0.975112	val-auc:0.718456
[00:04:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 56 pruned nodes, max_depth=8
[14923]	train-auc:0.975117	val-auc:0.718471
[00:04:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 52 pruned nodes, max_depth=8
[14924]	train-auc:0.975123	val-auc:0.718474
[00:04:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 42 pruned nodes, max_depth=8
[14925]	train-auc:0.975124	val-auc:0.718473
[00:04:43] C:\Users\

[00:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 54 pruned nodes, max_depth=8
[14963]	train-auc:0.975284	val-auc:0.718532
[00:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 42 pruned nodes, max_depth=8
[14964]	train-auc:0.975287	val-auc:0.718544
[00:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 50 pruned nodes, max_depth=8
[14965]	train-auc:0.975293	val-auc:0.718545
[00:04:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 50 pruned nodes, max_depth=8
[14966]	train-auc:0.975292	val-auc:0.718541
[00:04:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 58 pruned nodes, max_depth=8
[14967]	train-auc:0.975299	val-auc:0.718531
[00:04:54] C:\Users\